In [ ]:
#Copyright © 2024 LOCBP @ University of Zürich
#Distributed under MIT license

APPEND_JOB = False
"""
INPUT PARAMETERS
"""
"""ProteinMPNN"""
NAME = "" #if empty, the name will be the PDB #No whitespaces
PDB = "" #pdb (upload not implemented) or pdb folder containing folded proteins
NUM_SEQ = 1
FIXED = "" #pymol-formatted selection e.g. "10-15+17+20-54". Incompatible with pLDDT_THRESHOLD when a folder is given as input
FIXED_CHAIN = "A" #To be implemented: possibility to fix more than one chain
pLDDT_THRESHOLD = 100 #Will consider residues with pLDDT >= threshold as fixed. Only if pLDDT is stored in b values of the protein, so input must be from AlphaFold or OmegaFold
#Advanced
SAMPLING_T = "0.1"
"""FOLD"""
FOLD = "AF" #Choices are AF for AlphaFold and OF for OmegaFold
#AlphaFold
#Advanced
NUM_RELAX = 0 #How many of the best-ranked models do you want to relax with amber?
NUM_RECYCLE = 3 #Default (and recommended) is 3
RAND_SEED = 0
ONLY_FIRST = True #Only compare the best folding of each sequence generated
#OmegaFold
MODEL = 2 #Model 2 can only be used with V100-32GB or A100-80GB
#Ranking
METRIC = "pLDDT" #"pLDDT" or "pTM" or "RMSD". pTM not available in omegafold
RMSD_ALIGNMENT = 'cealign' #'cealign' or 'align' or 'super' or 'fit'
PYMOL_BEST = 10 #Create a pymol session contaning the N best models aligned with the original protein and colored by pLDDT
#DNA
DNA=True #Generates DNA sequences for Homo Sapiens and E coli

"""
CODE
"""
#Quick refinement of input
ONE_JOB = not APPEND_JOB
if FOLD == "AF": FOLD = "AlphaFold"
elif FOLD == "OF": FOLD = "OmegaFold"
if FIXED == "":
    FIXED = "-"
    
import os,shutil #WE USE ABSOLUTE PATHS
#CUDA
ENVIRONMENT = "ProteinEnv"
#General
MODELS_OUTPUT_FOLDER = "scratch" #Where to store output (subfolders will be created inside)
INSTALLATION_FOLDER = "data"
#GENERAL FOLDERS
NOTEBOOKS_FOLDER = os.getcwd()
PIPELINES_FOLDER = os.path.join(NOTEBOOKS_FOLDER,"Pipelines")
PDBs_FOLDER = os.path.join(NOTEBOOKS_FOLDER,"PDBs")
HELP_FOLDER = os.path.join(NOTEBOOKS_FOLDER,"HelpScripts")
USER_NAME = os.path.basename(os.path.dirname(NOTEBOOKS_FOLDER))
HOME_FOLDER = f"/home/{USER_NAME}"
DATA_FOLDER = f"/data/{USER_NAME}"
SCRATCH_FOLDER = f"/scratch/{USER_NAME}"
#MODIFIABLE FOLDERS
INSTALLATION_FOLDER = DATA_FOLDER
MODELS_OUTPUT_FOLDER = os.path.join(SCRATCH_FOLDER,"ProteinOutput")
if not os.path.exists(MODELS_OUTPUT_FOLDER):
    os.mkdir(MODELS_OUTPUT_FOLDER)
#MODELS
PMPNN_FOLDER = os.path.join(INSTALLATION_FOLDER,"ProteinMPNN")
COLAB_FOLD_FOLDER = os.path.join(INSTALLATION_FOLDER,"localcolabfold")
OMEGA_FOLDER = os.path.join(INSTALLATION_FOLDER,"OmegaFold")
OUT_FOLDER = os.path.join(MODELS_OUTPUT_FOLDER,"ProteinMPNN")
if not os.path.exists(OUT_FOLDER):
    os.mkdir(OUT_FOLDER)

#This will be used throughout to generate file and directory names to avoid overriding old outputs
def unique_name(directory,root,ext = "",fullpath=0,w=3):   
    i = 1     
    u_name = root + "_" + "{:0>{width}}".format(i, width=w)
    while os.path.exists(os.path.join(directory,u_name+ext)):
        i += 1
        u_name = root + "_" + "{:0>{width}}".format(i, width=w)
    if fullpath: return os.path.join(directory, u_name + ext)
    return u_name + ext

"""
SET JOB NAME AND CREATE OUTPUT DIRECTORIES
Data are stored in the "PMPNN_output" folder inside the data folder
Also, a folder containg all the results is created inside that folder
"""
JOB_BASE = NAME if NAME != "" else os.path.splitext(os.path.basename(PDB))[0]
JOB = unique_name(OUT_FOLDER,JOB_BASE)
JOB_FOLDER = os.path.join(OUT_FOLDER,JOB)
os.mkdir(JOB_FOLDER)
RUNTIME_FOLDER = os.path.join(JOB_FOLDER,"RunTime")
if not os.path.exists(RUNTIME_FOLDER):
    os.mkdir(RUNTIME_FOLDER)
#Output from console will also be redirected to log files
pmpnn_log_file = os.path.join(JOB_FOLDER,"_pmpnn.log")
fold_log_file = os.path.join(JOB_FOLDER,f"_{FOLD}.log")
ranking_log_file = os.path.join(JOB_FOLDER,"_ranking.log")
dna_log_file = os.path.join(JOB_FOLDER,"_dna.log")

"""ProteinMPNN"""
#Parse folder for multiple pdb input
parsed_pdbs_jsonl_file = os.path.join(RUNTIME_FOLDER,f"{JOB}_parsed_pdbs.jsonl")
parse_py_file = os.path.join(PMPNN_FOLDER,"helper_scripts/parse_multiple_chains.py")
fixed_jsonl_file = os.path.join(RUNTIME_FOLDER,f"{JOB}_fixed_pos.jsonl")
sele_csv_file = os.path.join(RUNTIME_FOLDER,"pmpnn_sele.csv")

pdb_temp = PDB if PDB.endswith(".pdb") else PDB + ".pdb"
pdb_temp_file = os.path.join(PDBs_FOLDER,pdb_temp)
if os.path.exists(pdb_temp_file): #PDB is a file
    pdb_file = os.path.join(RUNTIME_FOLDER,pdb_temp)
    shutil.copy(pdb_temp_file,pdb_file)
else: #otherwise it is a folder
    pdb_folder = os.path.join(os.getcwd(),PDB)
    if os.path.exists(pdb_folder):
        pdb_files = [pdb for pdb in os.listdir(pdb_folder) if pdb.endswith(".pdb")]
        for pdb_file in pdb_files:
            new_pdb_file = os.path.join(RUNTIME_FOLDER,pdb_file)
            shutil.copy(os.path.join(pdb_folder,pdb_file),new_pdb_file)

#IMPLEMENTATION OF FIXED POSITIONS
fixed_py_file = os.path.join(HELP_FOLDER,"make_fixed_dict.py")

#Set options
pmpnn_options = f"--num_seq_per_target {NUM_SEQ}"
pmpnn_options += f" --sampling_temp {SAMPLING_T}"
pmpnn_py_file = os.path.join(PMPNN_FOLDER,"protein_mpnn_run.py")

pmpnn_sh_file = unique_name(RUNTIME_FOLDER,"pmpnn",".sh",1)
pmpnn_cmd = f"""
echo Determining fixed positions
python {fixed_py_file} {RUNTIME_FOLDER} norfd {pLDDT_THRESHOLD} {FIXED} {FIXED_CHAIN} {fixed_jsonl_file} {sele_csv_file}
echo Parsing multiple pbds 
python {parse_py_file} --input_path {RUNTIME_FOLDER} --output_path {parsed_pdbs_jsonl_file}
echo Running model
python {pmpnn_py_file} --jsonl_path {parsed_pdbs_jsonl_file} --fixed_positions_jsonl {fixed_jsonl_file} --out_folder {JOB_FOLDER} {pmpnn_options}
"""
with open(pmpnn_sh_file,'w') as pmpnn_sh:
    pmpnn_sh.write(pmpnn_cmd)
os.chmod(pmpnn_sh_file, 0o755) #Give execution rights

"""FOLD"""
FOLD_OUT_FOLDER = os.path.join(JOB_FOLDER,FOLD)
if not os.path.exists(FOLD_OUT_FOLDER):
    os.mkdir(FOLD_OUT_FOLDER)
PMPNN_FA_FOLDER = os.path.join(JOB_FOLDER,"seqs") #This is the folder where pmpnn outputs fasta files
queries_csv_file = os.path.join(RUNTIME_FOLDER,"queries.csv") #These will contain the queries
queries_fasta_file = os.path.join(JOB_FOLDER,"queries.fasta")
fa_to_csv_fasta_py_file = os.path.join(HELP_FOLDER,"fa_to_csv_fasta.py")

fold_sh_file = ""
if FOLD == "AlphaFold":
    af2_options = ""
    if NUM_RELAX > 0: af2_options += f" --amber --use-gpu-relax --num-relax {NUM_RELAX}" #assumed to run on GPU
    if NUM_RECYCLE != 3: af2_options += f" --num-recycle {NUM_RECYCLE}"
    if RAND_SEED != 0: af2_options += f" --random-seed {RAND_SEED}"

    colabfold_batch_file = os.path.join(COLAB_FOLD_FOLDER,"colabfold-conda/bin/colabfold_batch")
    fold_sh_file = unique_name(RUNTIME_FOLDER,"alphafold",".sh",1)
    af2_cmd = f"""
    echo Generating queries.csv file from .fa output
    python {fa_to_csv_fasta_py_file} {PMPNN_FA_FOLDER} {queries_csv_file} {queries_fasta_file}
    echo Initializing model...
    {colabfold_batch_file} {queries_csv_file} {FOLD_OUT_FOLDER} {af2_options}
    """
    with open(fold_sh_file,'w') as af2_sh:
        af2_sh.write(af2_cmd)
elif FOLD == "OmegaFold":
    of_options = ""
    of_options += f" --model {MODEL}"
    weights_pt = ["","release1.pt","release2.pt"][MODEL]
    weights_pt_file = os.path.join(OMEGA_FOLDER,weights_pt)
    of_options += f" --weights_file {weights_pt_file}"
    fold_sh_file = unique_name(RUNTIME_FOLDER,"omegafold",".sh",1)
    of_cmd = f"""
    echo Generating queries.csv file from .fa output
    python {fa_to_csv_fasta_py_file} {PMPNN_FA_FOLDER} {queries_csv_file} {queries_fasta_file}
    echo Initializing model...
    omegafold {queries_fasta_file} {FOLD_OUT_FOLDER} {of_options}
    """
    with open(fold_sh_file,'w') as of_sh:
        of_sh.write(of_cmd)
os.chmod(fold_sh_file, 0o755) #Give execution rights

"""RANK EVERYTHING"""
rank_pdb = pdb_file if pdb_file != "" else "-"
rank_output_csv_file = os.path.join(JOB_FOLDER,JOB_BASE+".csv")
rank_best_fasta_file = os.path.join(JOB_FOLDER,JOB_BASE+f"_Best{PYMOL_BEST}.fasta")
rank_pse_file = os.path.join(JOB_FOLDER,JOB_BASE+f"_{FOLD}.pse")
rank_py_file = os.path.join(HELP_FOLDER,f"rank_{FOLD}.py")
rank_cmd = f"python {rank_py_file} {fold_log_file} {queries_csv_file} {NUM_SEQ} {sele_csv_file} {FOLD_OUT_FOLDER} {rank_pdb} {rank_output_csv_file} {METRIC} {RMSD_ALIGNMENT} {rank_pse_file} {PYMOL_BEST} {rank_best_fasta_file} {ONLY_FIRST}"
rank_sh_file = unique_name(RUNTIME_FOLDER,f"rank_{FOLD}",".sh",1)
with open(rank_sh_file,'w') as rank_sh:
    rank_sh.write(rank_cmd)
os.chmod(rank_sh_file, 0o755) #Give execution rights

"""DNA"""
DNA_FOLDER=os.path.join(JOB_FOLDER,"DNA")
if not os.path.exists(DNA_FOLDER):
    os.mkdir(DNA_FOLDER)
dna_prefix = os.path.join(DNA_FOLDER,JOB_BASE)
dna_encoder_py_file = os.path.join(HELP_FOLDER,"dna_encoder.py")
dna_encoder_cmd = f"python {dna_encoder_py_file} {rank_best_fasta_file} {dna_prefix}"
dna_encoder_sh_file = os.path.join(RUNTIME_FOLDER,"dna_encoder.sh")
with open(dna_encoder_sh_file,'w') as dna_encoder_sh:
    dna_encoder_sh.write(dna_encoder_cmd)
os.chmod(dna_encoder_sh_file, 0o755) #Give execution rights

"""COMBINE ALL IN A UNIQUE PIPELINE"""
pipeline_sh_file = unique_name(RUNTIME_FOLDER,"pipeline",".sh",1)
print_options_cmd = f"""
echo Pipeline: {os.path.basename(pipeline_sh_file)}
echo
echo Name: {NAME}
echo PDB: {PDB}


echo ProteinMPNN
echo   NUM SEQUENCES PER TARGET: {NUM_SEQ}
echo   FIXED: {FIXED}
echo   FIXED CHAIN: {FIXED_CHAIN} 
echo   pLDDT THR: {pLDDT_THRESHOLD}
echo   SAMPLING T: {SAMPLING_T}
echo
echo FOLD MODEL: {FOLD}
echo   AlphaFold
echo     NUM RELAX: {NUM_RELAX}
echo     NUM RECYCLE: {NUM_RECYCLE}
echo     RANDOM GENERATOR SEED: {RAND_SEED}
echo   OmegaFold
echo     MODEL: {MODEL}
echo
echo "Alignment algorithm: {RMSD_ALIGNMENT}"
"""
pipeline_cmd = print_options_cmd + f"""
source activate {ENVIRONMENT}
echo
echo ProteinMPNN
{pmpnn_sh_file} | tee {pmpnn_log_file}
echo
echo {FOLD}
{fold_sh_file} | tee {fold_log_file}
echo 
echo Ranking...
{rank_sh_file} | tee {ranking_log_file}"""
if DNA: pipeline_cmd += f"""
echo
echo DNA
{dna_encoder_sh_file} | tee {dna_log_file}
"""
pipeline_cmd += f"""
echo 
echo Job done
echo {JOB_FOLDER}
"""
with open(pipeline_sh_file,'w') as pipeline_sh:
    pipeline_sh.write(pipeline_cmd)
os.chmod(pipeline_sh_file, 0o755) #Give execution rights

batch_sh_file = os.path.join(PIPELINES_FOLDER,"pmpnn_batch.sh")
if not ONE_JOB and os.path.exists(batch_sh_file):
    with open(batch_sh_file,"r") as pmpnn_batch_sh:
        previous_pipelines = pmpnn_batch_sh.readlines()
    with open(batch_sh_file,"w") as pmpnn_batch_sh:
        pmpnn_batch_sh.writelines(previous_pipelines)
        pmpnn_batch_sh.write("\n")
        pmpnn_batch_sh.write(pipeline_sh_file)  
else:      
    with open(batch_sh_file,"w") as pmpnn_batch_sh:
        pmpnn_batch_sh.write(pipeline_sh_file)  
os.chmod(batch_sh_file, 0o755) #Give execution rights

#write configuration file
config_txt_file = os.path.join(JOB_FOLDER,JOB_BASE+"_config.txt")
with open(config_txt_file,'w') as config_txt:
    config_txt.write(print_options_cmd)

print("Run using next cell")
print(f"Single job:\n{pipeline_sh_file}")
print(f"Batch:\n{batch_sh_file}")
print(f"\nOutput of this pipeline will be in:\n{JOB_FOLDER}")

In [ ]:
%%bash
/home/$USER/ProteinNotebooks/Pipelines/pmpnn_batch.sh

In [ ]:
"""
MAKE SLURM FILE
"""
PACKAGE_MANAGER = "mamba"
GPU = "A100" 
"""
GPUs available are:
T4      0.022 CHF/h
V100    0.057 CHF/h
A100    0.081 CHF/h
These can be allocated by setting GPU = "T4", "V100", or "A100"
By setting GPU="gpu", the first available will be used
By setting GPU="high-memory", either v100-32gb or a100 are selected. 
Important: OmegaFold model 2 only works with high-memory GPUs. Sometimes it fails with 32G as well, so it is safer to use A100
"""

slurm_file = unique_name(PIPELINES_FOLDER,"pmpnn_slurm",".sh",1)
with open(batch_sh_file,"r") as rfd_batch_sh:
    all_pipelines = rfd_batch_sh.readlines()
with open(slurm_file,"w") as slurm_bash:
    slurm_bash.write("""# Check if nvidia-smi is available
if ! command -v nvidia-smi &> /dev/null
then
    echo "Could not load GPU correctly: nvidia-smi could not be found"
    exit
fi
gpu_type=$(nvidia-smi --query-gpu=gpu_name --format=csv,noheader)
echo "GPU Type: $gpu_type"\n
""")
    slurm_bash.write(f"module load {PACKAGE_MANAGER}\n")
    slurm_bash.writelines(all_pipelines)
os.chmod(slurm_file, 0o755)

if GPU == "high-memory":
    gpu_line = "#SBATCH --gpus=1\n#SBATCH --constraint=\"GPUMEM32GB|GPUMEM80GB\""
elif GPU == "gpu":
    gpu_line = "#SBATCH --gpus=1"
else:
    gpu_line = f"#SBATCH --gpus={GPU}:1"


job_comps=JOB_FOLDER.split('/')
job_name=f"{job_comps[-2]}: {job_comps[-1]}"

print(f"""
ScienceApps > Jobs > JobComposer > New Job > From Default Template    
Edit Job name from Job Options. Suggested:

{job_name}       

Replace job.sh (Open in Editor) with the following (adapt required time hh:mm:ss) then Save:
      
#!/usr/bin/bash
{gpu_line}
#SBATCH --mem=7800
#SBATCH --time=23:59:00
#SBATCH --output=job.out      
{slurm_file}
""")

In [ ]:
"""
Conventions for names
Paths to files end with _type_file
Paths to folders end with _FOLDER
Opened files .type end with _type
Content of .sh end with _cmd
Content of .py end with _script
"""

"""
Cleanup instructions
Delete data/bash_files
Delete outputs folder: contains log files
"""